In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-06-29 22:00:49--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.69MB/s    in 0.2s    

2020-06-29 22:00:50 (4.69 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmznReviews1ETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
music_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"),sep='\t', header=True, inferSchema=True)

music_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...|2015-08-31 00:00:00|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...| 

In [5]:
# Finding the number of records in the dataset
music_data_df.count()

4751577

In [6]:
music_data_cleaned_df = music_data_df.dropna()
music_data_cleaned_df.count()

4750852

In [7]:
# counting the distinct customers

customers_data = music_data_cleaned_df.groupBy("customer_id").count()
customers_data.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|     651796|    2|
|   44979559|    1|
|   45632184|    1|
|   15232809|    1|
|   17067926|    1|
|   26163929|    1|
|   20587872|    2|
|   14230926|    3|
|   52081222|    2|
|   13382150|    1|
|   13517415|    1|
|   19324145|   33|
|   46311034|    2|
|   39430203|  113|
|    5573238|    1|
|   53095124|   11|
|     172523|    1|
|     283137|    1|
|    1356209|    1|
|   28074100|    2|
+-----------+-----+
only showing top 20 rows



In [8]:
# renaming the 'count' column
customers_data = customers_data.withColumnRenamed("count","customer_count")
customers_data.show()
type(customers_data)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     651796|             2|
|   44979559|             1|
|   45632184|             1|
|   15232809|             1|
|   17067926|             1|
|   26163929|             1|
|   20587872|             2|
|   14230926|             3|
|   52081222|             2|
|   13382150|             1|
|   13517415|             1|
|   19324145|            33|
|   46311034|             2|
|   39430203|           113|
|    5573238|             1|
|   53095124|            11|
|     172523|             1|
|     283137|             1|
|    1356209|             1|
|   28074100|             2|
+-----------+--------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [9]:
customers_data.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [10]:
# changing the datatype of customer_count to interger
customers_data= customers_data.withColumn("customer_count", customers_data.customer_count.cast("int"))

In [11]:
customers_data.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [13]:
# Create products dataframe 
products_data = music_data_cleaned_df.groupBy('product_id','product_title').count()
products_data = products_data.select(['product_id','product_title'])

products_data.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001FOJ76S|         The Promise|
|B000NJWSE4|                Free|
|B00002CF4Z|The Best of Steve...|
|B000CQM4T2|        Live Trucker|
|B000002QG8|            Float On|
|B003JYOFIW|Something for Eve...|
|B004MY64NM|             Manhole|
|B007I8TCD8|           Do Things|
|B001CY2ELQ|               Lenka|
|B00BQ1DBH6|The Best From The...|
|B00VXGTJMU|   Django and Jimmie|
|B00MI9OZ0W|BAYONETTA 2 Origi...|
|B003DC881A|   Chamberlain Waits|
|B000003MZG|            Africano|
|B00005QIUW|Christmas Through...|
|B000WSTAEC|Haendel: La Passi...|
|B002PXJZIS|Dont Stop Believi...|
|B00U646N7E|Directions Home (...|
|B0007MR1L2|Hummel: Septet, O...|
|B00AK77XUS|     The Flower Lane|
+----------+--------------------+
only showing top 20 rows



In [14]:
products_data.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [15]:
# Create review_id_table dataframe 
review_id_table = music_data_cleaned_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3LI5TRP3YIDQL|   10140119|B00TXH4OLC|     384427924|2015-08-31 00:00:00|
|R3LGC3EKEG84PX|   27664622|B00B6QXN6U|     831769051|2015-08-31 00:00:00|
| R9PYL3OYH55QY|   45946560|B001GCZXW6|      14067376|2015-08-31 00:00:00|
|R3PWBAWUS4NT0Q|   15146326|B000003EK6|     566295619|2015-08-31 00:00:00|
|R15LYP3O51UU9E|   16794688|B00N1F0BKK|     210426072|2015-08-31 00:00:00|
|R1AD7L0CC3DSRI|   32203364|B00V7KAO7Q|     360249504|2015-08-31 00:00:00|
|R32FE8Y45QV434|    1194276|B000094Q4P|      20972283|2015-08-31 00:00:00|
|R3NM4MZ4XWL43Q|   45813052|B00JMK0P1I|     378049560|2015-08-31 00:00:00|
|R3H4FXX6Q7I37D|   12795687|B008OW1S3O|     141620702|2015-08-31 00:00:00|
|R30L5PET7LFFDC|   36673840|B00VI2L3L4|     920557135|2015-08-31 00:00:00|
|  REFRE1LEKLAF|   494535

In [16]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [17]:
# changing the datatype of review_date to date
review_id_table=review_id_table.withColumn("review_date", review_id_table.review_date.cast("date"))

In [18]:
# Create vine_table dataframe 
vine_table = music_data_cleaned_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|
|R3LGC3EKEG84PX|          5|            0|          0|   N|
| R9PYL3OYH55QY|          5|            0|          1|   N|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|
|R15LYP3O51UU9E|          5|            0|          0|   N|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|
|R32FE8Y45QV434|          5|            0|          0|   N|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|
|R30L5PET7LFFDC|          5|            1|          1|   N|
|  REFRE1LEKLAF|          4|            1|          1|   N|
|R3JTJ5EQN74E9H|          5|            0|          0|   N|
|R1W2F091LCOAW5|          5|            0|          0|   N|
| RYUMFQRRB1FNM|          3|            

In [19]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [20]:
# changing the datatype of star_rating to integer
vine_table=vine_table.withColumn("star_rating", vine_table.star_rating.cast("int"))

In [21]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [23]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql:// :5432/Amzn_music_reviews"
config = {"user":<user>, 
          "password":<password>, 
          "driver":"org.postgresql.Driver"}

In [24]:
# Write DataFrame to customers table in RDS

customers_data.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)



In [25]:
sc= spark.sparkContext
from pyspark.sql import SQLContext
sqlContext= SQLContext(sc)
sqlContext.read.jdbc(url=jdbc_url, table='products', properties=config)

DataFrame[product_id: string, product_title: string]

In [26]:
# Write DataFrame to products table in RDS

products_data.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [27]:
# Write DataFrame to review_id_table  in RDS

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [28]:
# Write DataFrame to vine_table  in RDS

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)